In [26]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import shortest_path

def readInput(input):
    A = []
    lines = input.split("\n")
    for line in lines:
        row = []
        for n in line:
            row.append(int(n))
        A.append(row)
    return A

#Get the path, if we want it
#From https://stackoverflow.com/questions/53074947/examples-for-search-graph-using-scipy/53078901
def get_path(Pr, i, j):
    path = [j]
    k = j
    while Pr[i, k] != -9999:
        path.append(Pr[i, k])
        k = Pr[i, k]
    return path[::-1]


def getAdjGraph(A):
    r, s = len(A), len(A[0])
    y = 0
    x = 0
    #M will be the indcidence matrix of the nodes.
    #Apply risk to entering this node
    M = np.zeros((r*s,r*s),dtype=np.uint8)
    for cnt in range(0,r*s):
        risk = A[y][x]
        if x+1 < s:
            M[cnt+1][cnt] = risk
        if x-1 >= 0:
            M[cnt-1][cnt] = risk
        if y+1 < r:
            M[cnt+s][cnt] = risk
        if y-1 >= 0:
            M[cnt-s][cnt] = risk
        x += 1
        if x%s == 0:
            x=0
            y+=1
    return M

Inp = readInput(input)
A = getAdjGraph(Inp)
C = csr_matrix(A)
D, Pr = shortest_path(csgraph=C, directed=True, method='D', return_predecessors=True)
print(get_path(Pr,0,-1))
print(D[0,-1])

[0, 10, 20, 21, 22, 23, 24, 25, 26, 36, 37, 47, 48, 58, 68, 78, 88, 89, -1]
40.0


In [27]:
import sys 
import numpy as np

#Graph gets too big to use adjacency and built in Djikstra, instead we code our own version to take advantage of the matrix graph structure
#Following is modified wikipedia pseudocode
#Even simplified, it can take a while to run this

def readInput(input):
    A = []
    lines = input.split("\n")
    for line in lines:
        row = []
        for n in line:
            row.append(int(n))
        A.append(row)
    return A

def getMinVertex(Q, dist):
    minVertex = -1
    minDistance = sys.maxsize
    for vertex in Q:
        if dist[vertex] < minDistance:
            minVertex = vertex
            minDistance = dist[vertex]
            
    return minVertex

def djikstraUpdate(dist, prev, u, v, Aval):
    alt = dist[u] + Aval
    if alt < dist[v]:
        dist[v] = alt
        prev[v] = u
    return


def djikstra(A):
    M,N = len(A), len(A[0])
    dist = [sys.maxsize]*M*N
    prev = [-sys.maxsize]*M*N
    dist[0] = 0
    
    Qdict = {}
    for i in range(0,M*N):
        Qdict[i] = True

    while len(Qdict.keys()) != 0:
        u = getMinVertex(Qdict.keys(),dist)
        x = int(u%N)
        y = int((u-x)/N)
        del Qdict[u]
        
        if x+1 < N:
            djikstraUpdate(dist, prev, u, u+1, A[y][x+1])          
        if x-1 >= 0:
            djikstraUpdate(dist, prev, u, u-1, A[y][x-1])          
        if y+1 < M:
            djikstraUpdate(dist, prev, u, u+N, A[y+1][x])
        if y-1 >= 0:
            djikstraUpdate(dist, prev, u, u-N, A[y-1][x])        
    return dist, prev

def get_path(Pr, j):
    path = [j]
    k = j
    while Pr[k] != -sys.maxsize:
        path.append(Pr[k])
        k = Pr[k]
    return path[::-1]

#Inp = readInput(input)
#D, Pr = djikstra(Inp)
#print(get_path(Pr,-1))
#print(D[-1])

#get first large grid tile
Inp = np.array(readInput(input))
factor = 5
tiles = []
for cnt in range(0,factor):
    tiles.append((Inp+cnt-1)%9+1)
rowTile = np.concatenate(tiles, axis=1)

#get remaining tiles iterated
tiles = []
for cnt in range(0,factor):
    tiles.append((rowTile+cnt-1)%9+1)
largeTile = np.concatenate(tiles, axis=0)

D, Pr = djikstra(largeTile)
print(get_path(Pr,-1))
print(D[-1])


[0, 50, 100, 101, 102, 103, 104, 105, 106, 156, 157, 207, 208, 209, 259, 260, 261, 262, 263, 313, 314, 315, 316, 317, 318, 319, 369, 419, 420, 421, 422, 472, 473, 474, 524, 574, 575, 625, 626, 676, 726, 776, 826, 876, 926, 976, 1026, 1027, 1077, 1078, 1128, 1178, 1179, 1180, 1181, 1182, 1232, 1282, 1332, 1333, 1383, 1384, 1385, 1386, 1387, 1437, 1487, 1488, 1538, 1588, 1638, 1688, 1689, 1690, 1691, 1741, 1791, 1841, 1842, 1843, 1893, 1943, 1944, 1945, 1995, 1996, 1997, 2047, 2097, 2147, 2148, 2198, 2248, 2298, 2348, 2398, 2448, 2498, -1]
315


In [25]:
input = """1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581"""

In [ ]:
#This is an attempt to get the library code to work with the adjacency matrix. Unfortunately, this code while it works in teh example case, times out for the big input
def getShortestPaths(largeGrid):
    A = getAdjGraph(largeGrid)
    C = csr_matrix(A)
    D, Pr = shortest_path(csgraph=C, directed=True, method='D', return_predecessors=True)
    return D, Pr


#get first large grid tile
Inp = np.array(readInput(input))
factor = 5
tiles = []
for cnt in range(0,factor):
    tiles.append((Inp+cnt-1)%9+1)
rowTile = np.concatenate(tiles, axis=1)
R = len(rowTile[0])
numNodes = 1+factor*R
metaAdjMatrix = np.zeros((numNodes,numNodes))

#get distance from start to next level
D,Pr = getShortestPaths(rowTile)
for cnt in range(0,R):
    metaAdjMatrix[0][cnt+1] = D[0,-R+cnt]
#print(D[0,-R:])   

#get remaining rows and distances
for i in range(1,factor):
    rowTile = ((rowTile)%9+1)
    D,Pr = getShortestPaths(rowTile)
    
    #calculate end of one tile to end of next tile distances
    for fromCnt in range(0,R):
        for toCnt in range(0,R):
            fromLoc = 1+(i-1)*R+fromCnt
            toLoc = 1+i*R+toCnt
            metaAdjMatrix[fromLoc][toLoc] = rowTile[0][fromCnt] + D[fromCnt, -R+toCnt]

#Rowtile debugging
#print(rowTile)
            
#Calculate shortest path on meta graph
metaA = csr_matrix(metaAdjMatrix)
metaD, metaPr = shortest_path(csgraph=metaA, directed=True, method='FW', return_predecessors=True)
print(get_path(metaPr,0,-1))
print(metaD[0,-1])

